In [1]:
import matplotlib.pyplot as plt
from pyspark.sql.functions import collect_list
from pyspark.sql.types import StructField, StructType, StringType, LongType
import pandas as pd

In [2]:
# Import dataframe 
Iowa = spark.read\
.format("csv")\
.option("header", "true")\
.option("nullValue", "NA")\
.option("delimiter", "\t")\
.option("inferSchema", "true")\
.load("gs://pstat135-mh-project/data/VM2Uniform--IA--2021-03-04/VM2Uniform--IA--2021-03-04.tab")


In [3]:
Iowa.printSchema()

root
 |-- SEQUENCE: integer (nullable = true)
 |-- LALVOTERID: string (nullable = true)
 |-- Voters_Active: string (nullable = true)
 |-- Voters_StateVoterID: integer (nullable = true)
 |-- Voters_CountyVoterID: string (nullable = true)
 |-- VoterTelephones_LandlineAreaCode: integer (nullable = true)
 |-- VoterTelephones_Landline7Digit: string (nullable = true)
 |-- VoterTelephones_LandlineFormatted: string (nullable = true)
 |-- VoterTelephones_LandlineUnformatted: long (nullable = true)
 |-- VoterTelephones_LandlineConfidenceCode: integer (nullable = true)
 |-- VoterTelephones_CellPhoneOnly: boolean (nullable = true)
 |-- VoterTelephones_CellPhoneFormatted: string (nullable = true)
 |-- VoterTelephones_CellPhoneUnformatted: long (nullable = true)
 |-- VoterTelephones_CellConfidenceCode: integer (nullable = true)
 |-- Voters_FirstName: string (nullable = true)
 |-- Voters_MiddleName: string (nullable = true)
 |-- Voters_LastName: string (nullable = true)
 |-- Voters_NameSuffix: string

In [43]:
# choose variables to include in dataframe 
Iowa_df = Iowa[['County',
             'Parties_Description',
             "Residence_Addresses_Property_LandSq_Footage",
             "Residence_Addresses_Property_HomeSq_Footage",
             "CommercialData_LandValue",
             "Voters_Gender",
             "Voters_Age",
             "CommercialData_AreaMedianEducationYears",
             "CommercialDataLL_Home_Owner_Or_Renter",
             "CommercialData_EstimatedAreaMedianHHIncome",
             "CommercialData_AreaMedianHousingValue"]]

# transform to pandas dataframe 
Iowa_df = Iowa_df.toPandas()



In [44]:
# rename columns 
Iowa_df = Iowa_df.rename(columns = {"Residence_Addresses_Property_LandSq_Footage" : "Property_LandSq_Footage",
                                      "Residence_Addresses_Property_HomeSq_Footage" : "Property_HomeSq_Footage",
                                      "CommercialData_LandValue" : "LandValue",
                                      "CommercialData_AreaMedianEducationYears" : "AreaMedianEducationYears",
                                      "CommercialDataLL_Home_Owner_Or_Renter" : "Home_Owner_Or_Renter",
                                      "CommercialData_EstimatedAreaMedianHHIncome" : "EstimatedAreaMedianHHIncome",
                                      "CommercialData_AreaMedianHousingValue" : "AreaMedianHousingValue"})
Iowa_df.head()

,County,Parties_Description,Property_LandSq_Footage,Property_HomeSq_Footage,LandValue,Voters_Gender,Voters_Age,AreaMedianEducationYears,Home_Owner_Or_Renter,EstimatedAreaMedianHHIncome,AreaMedianHousingValue
0,ADAIR,Democratic,342000.0,3500.0,None,F,35.0,12.0,None,$66266,$104166
1,ADAIR,Non-Partisan,342000.0,3500.0,None,M,40.0,12.0,None,$66266,$104166
2,ADAIR,Non-Partisan,1698000.0,1600.0,$39000,F,32.0,12.0,None,$66266,$104166
3,ADAIR,Non-Partisan,50000.0,2300.0,None,F,34.0,12.0,Likely Homeowner,$66266,$104166
4,ADAIR,Democratic,1555000.0,2800.0,$35000,F,31.0,12.0,None,$69948,$91666


In [45]:
# percentage of data missing for each variable
percent_missing = Iowa_df.isnull().sum() * 100 / len(Iowa_df)

percent_missing

County                          0.000000
Parties_Description             0.000000
Property_LandSq_Footage        16.660255
Property_HomeSq_Footage        18.341540
LandValue                      31.893801
Voters_Gender                   0.000000
Voters_Age                      0.050372
AreaMedianEducationYears        3.614241
Home_Owner_Or_Renter           27.414945
EstimatedAreaMedianHHIncome     3.614241
AreaMedianHousingValue          3.619193
dtype: float64

    The EstimatedAreaMedianHHIncome, AreaMedianHousingValue, AreaMedianEducationYears and Voters_age variables have a very low proportion of missing values.We can proceed by changing the variable types from object to float and filling in the data using the fillna method. 

In [46]:
# remove dollar signs from estimated median household income and convert to numeric
Iowa_df["EstimatedAreaMedianHHIncome"] = Iowa_df["EstimatedAreaMedianHHIncome"].str.replace("$"," ")
Iowa_df["AreaMedianHousingValue"] = Iowa_df["AreaMedianHousingValue"].str.replace("$"," ")
Iowa_df["LandValue"] = Iowa_df["LandValue"].str.replace("$"," ")

# change types of variables 
Iowa_df["EstimatedAreaMedianHHIncome"] = pd.to_numeric(Iowa_df["EstimatedAreaMedianHHIncome"])
Iowa_df["AreaMedianEducationYears"] = pd.to_numeric(Iowa_df["AreaMedianEducationYears"])
Iowa_df["AreaMedianHousingValue"] = pd.to_numeric(Iowa_df["AreaMedianHousingValue"])
Iowa_df["LandValue"] = pd.to_numeric(Iowa_df["LandValue"])

Iowa_df.dtypes

/tmp/ipykernel_11534/3453683641.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  Iowa_df["EstimatedAreaMedianHHIncome"] = Iowa_df["EstimatedAreaMedianHHIncome"].str.replace("$"," ")
/tmp/ipykernel_11534/3453683641.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  Iowa_df["AreaMedianHousingValue"] = Iowa_df["AreaMedianHousingValue"].str.replace("$"," ")
/tmp/ipykernel_11534/3453683641.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  Iowa_df["LandValue"] = Iowa_df["LandValue"].str.replace("$"," ")


County                          object
Parties_Description             object
Property_LandSq_Footage        float64
Property_HomeSq_Footage        float64
LandValue                      float64
Voters_Gender                   object
Voters_Age                     float64
AreaMedianEducationYears       float64
Home_Owner_Or_Renter            object
EstimatedAreaMedianHHIncome    float64
AreaMedianHousingValue         float64
dtype: object

In [47]:
# fill missing values with mean of each column
Iowa_clean = Iowa_df

Iowa_clean["EstimatedAreaMedianHHIncome"] = Iowa_clean["EstimatedAreaMedianHHIncome"].fillna(Iowa_clean["EstimatedAreaMedianHHIncome"].mean())
Iowa_clean["AreaMedianHousingValue"] = Iowa_clean["AreaMedianHousingValue"].fillna(Iowa_clean["AreaMedianHousingValue"].mean())
Iowa_clean["AreaMedianEducationYears"] = Iowa_clean["AreaMedianEducationYears"].fillna(Iowa_clean["AreaMedianEducationYears"].mean())
Iowa_clean["Voters_Age"] = Iowa_clean["Voters_Age"].fillna(Iowa_clean["Voters_Age"].mean())


In [48]:

Iowa_clean.isnull().sum() * 100 / len(Iowa_clean)

County                          0.000000
Parties_Description             0.000000
Property_LandSq_Footage        16.660255
Property_HomeSq_Footage        18.341540
LandValue                      31.893801
Voters_Gender                   0.000000
Voters_Age                      0.000000
AreaMedianEducationYears        0.000000
Home_Owner_Or_Renter           27.414945
EstimatedAreaMedianHHIncome     0.000000
AreaMedianHousingValue          0.000000
dtype: float64

We can now drop rows that have missing values in multiple columns. 

In [49]:
# Keep only the rows with at least 9 non-NA values.
Iowa_clean = Iowa_clean.dropna(thresh = 9)

# drop rows with missing values in LandValue and Home owner or renter

Iowa_clean = Iowa_clean.dropna(subset = ["LandValue","Home_Owner_Or_Renter"],how = "all")


In [52]:
# Only 2.37% and 4.44% is missing for Land square footage and home square footage respectively so fill in with mean 
Iowa_clean["Property_LandSq_Footage"] = Iowa_clean["Property_LandSq_Footage"].fillna(Iowa_clean["Property_LandSq_Footage"].mean())
Iowa_clean["Property_HomeSq_Footage"] = Iowa_clean["Property_HomeSq_Footage"].fillna(Iowa_clean["Property_HomeSq_Footage"].mean())


In [64]:
# drop rows with any missing values in home owner or renter 
Iowa_clean = Iowa_clean.dropna(subset = ["Home_Owner_Or_Renter"])

Iowa_clean.isnull().sum() * 100 / len(Iowa_clean1)

# Only 16% of data is missing in Land Value so fill in using the mean 
Iowa_clean["LandValue"] = Iowa_clean["LandValue"].fillna(Iowa_clean["LandValue"].mean())

In [66]:
Iowa_clean.isnull().sum() * 100 / len(Iowa_clean)

County                         0.0
Parties_Description            0.0
Property_LandSq_Footage        0.0
Property_HomeSq_Footage        0.0
LandValue                      0.0
Voters_Gender                  0.0
Voters_Age                     0.0
AreaMedianEducationYears       0.0
Home_Owner_Or_Renter           0.0
EstimatedAreaMedianHHIncome    0.0
AreaMedianHousingValue         0.0
dtype: float64

    Our dataset is now clean 

In [69]:
# group by county and calculate median property square footage 

Iowa_grouped = Iowa_clean.groupby(by = ["County"]).aggregate({'Property_LandSq_Footage':'median'})

# Sort by descending order and reset index
Iowa_grouped = Iowa_grouped_county.sort_values(by = 'Property_LandSq_Footage', ascending = False).reset_index()

Iowa_grouped

,County,Property_LandSq_Footage
0,RINGGOLD,669000.0
1,MADISON,130000.0
2,VAN BUREN,114000.0
3,DAVIS,108500.0
4,ADAMS,90500.0
...,...,...
94,POLK,10000.0
95,BLACK HAWK,10000.0
96,SCOTT,9000.0
97,CERRO GORDO,9000.0


In [71]:
#extract counties
counties = Iowa_grouped["County"]

#retrieve top 5. and bottom 5 county names in terms of median square footage
top5 = counties.head(5)
bottom5 = counties.tail(5)

top = ["RINGGOLD", "MADISON", "VAN BUREN", "DAVIS", "ADAMS"]
bottom = ["POLK","BLACK HAWK","SCOTT", "CERRO GORDO","WOODBURY"]


In [72]:
#return top 5 and bottom 5 counties in terms of square footage from clean dataframe 
top5_df = Iowa_clean[Iowa_clean["County"].isin(top)]
top5_df


bottom5_df = Iowa_clean[Iowa_clean["County"].isin(bottom)]
bottom5_df

,County,Parties_Description,Property_LandSq_Footage,Property_HomeSq_Footage,LandValue,Voters_Gender,Voters_Age,AreaMedianEducationYears,Home_Owner_Or_Renter,EstimatedAreaMedianHHIncome,AreaMedianHousingValue
47228,BLACK HAWK,Non-Partisan,17000.000000,1600.000000,19000.000000,F,24.0,14.000000,Likely Homeowner,71521.000000,152646.000000
47232,BLACK HAWK,Republican,163680.050212,2618.871593,68000.000000,M,28.0,12.699648,Likely Homeowner,81539.393252,162983.144173
47240,BLACK HAWK,Non-Partisan,163680.050212,2618.871593,29000.000000,M,23.0,12.699648,Likely Renter,81539.393252,162983.144173
47241,BLACK HAWK,Democratic,10000.000000,1200.000000,20000.000000,M,31.0,12.000000,Likely Homeowner,65095.000000,148129.000000
47245,BLACK HAWK,Non-Partisan,27000.000000,3700.000000,28000.000000,M,50.0,12.000000,Likely Homeowner,70199.000000,165159.000000
...,...,...,...,...,...,...,...,...,...,...,...
2087265,WOODBURY,Republican,43000.000000,2300.000000,35604.293746,F,76.0,13.000000,Likely Homeowner,116744.000000,247916.000000
2087266,WOODBURY,Republican,43000.000000,2300.000000,35604.293746,M,79.0,13.000000,Likely Homeowner,116744.000000,247916.000000
2087268,WOODBURY,Republican,58000.000000,5800.000000,35604.293746,M,84.0,13.000000,Likely Homeowner,116744.000000,247916.000000
2087269,WOODBURY,Democratic,112000.000000,3500.000000,69000.000000,M,62.0,13.000000,Likely Homeowner,116744.000000,247916.000000
